In [434]:
import numpy as np
from pydub import AudioSegment

In [509]:
# --- Parameters for binomial distribution affecting the grid
# define whether the meter is duple or triple (0: duple, 1: triple)
UT = 0.24
# define whether the tactus is duple or triple (0: duple, 1: triple)
LT = 0.22
# define whether to continue generating tactus beat using current meter
An = 0.2
# define whether to continue with the tactus subdivision or change it
Bn = 0.4

# define inter-onset interval for tactus level (in ms)
ioi = 400
# define fluctuation in tactus interval when sampling
fluctuation = ioi/100

# --- Parameters for binomial distribution affecting the presence of onset on beat
# obs: ordered according the level of strength
MP = 0.8
UP = [0.2, 0.6, 0.5]
LP = [0.4, 0.7, 0.4]

In [510]:
# --- The generative process
# idea: sample meter_ and tactus_ from binomial distributions
n_bars = 16
tactus_beats = np.zeros((0,))
tatum_beats = np.zeros((0,))
meter_beats = np.zeros((0,))
for n in range(n_bars):
    # shall we change the meter?
    meter_switch = np.random.binomial(1, An)
    if meter_switch or not n:
        # sample first tactus interval from normal distribution with ioi as mean
        first_tactus_interval = np.random.normal(ioi, fluctuation)
        # sample to decide whether meter is duple of triple
        meter_ = np.random.binomial(1, UT)
        print('meter is %s' % ('duple' if not meter_ else 'triple'))
        n_tactus = 2 if not meter_ else 3
        
    # --- Meter level
    # sample the bar duration
    bar_duration = np.random.normal(first_tactus_interval, fluctuation, 1)*n_tactus
    meter_beats = np.append(meter_beats, bar_duration)
    
    # --- Tactus level
    # generate 2 or 3 tactus beat for bar (defined by meter_)
    generated_tactus_for_bar = np.random.normal(first_tactus_interval, fluctuation, n_tactus)
    # associate each tactus to the beat position (to be used when choosing probability)
    generated_tactus_for_bar = np.hstack(
        (
            # an array of indexes ranging from 1 to the no. of tactus
            np.arange(1, generated_tactus_for_bar.shape[0]+1).reshape(-1,1),
            generated_tactus_for_bar.reshape(-1,1),
            
        )
    )
    tactus_beats = np.append(tactus_beats, generated_tactus_for_bar)
    
    # --- Tatum level
    # sample to decide whether to change no. of tatum beats
    tatum_switch = np.random.binomial(1, Bn)
    if tatum_switch or not n:
        # sample to decide whether tactus subdivision is duple or triple
        tactus_subdivision_ = np.random.binomial(1, LT)
        print('subdivision is %s' % ('duple' if not tactus_subdivision_ else 'triple'))
        n_tatum = 2 if not tactus_subdivision_ else 3
        
    # tatum level: generate 2 or 3 tatum beat for each tactus (defined by tactus_)
    # obs: interval between tatum changes as n_tatum changes
    for m in range(n_tactus):
        generated_tatum_for_bar = np.random.normal(first_tactus_interval/n_tatum, fluctuation, n_tatum)
        generated_tatum_for_bar = np.hstack(
            (
                # an array of indexes ranging from 1 to the no. of tactus
                np.arange(1, generated_tatum_for_bar.shape[0]+1).reshape(-1,1),
                generated_tatum_for_bar.reshape(-1,1)
            )
        )
        tatum_beats = np.append(tatum_beats, generated_tatum_for_bar)
        
# reshape arrays into (idx, time_interval) 
tactus_beats = tactus_beats.reshape(-1, 2)
tatum_beats = tatum_beats.reshape(-1, 2)

# assert if we really have generated x bars
assert meter_beats.shape[0] == n_bars

meter is duple
subdivision is duple
meter is duple
meter is duple
subdivision is triple
meter is duple
subdivision is duple
meter is triple
subdivision is duple
meter is triple
meter is duple
subdivision is duple
meter is duple
subdivision is duple
subdivision is duple


In [511]:
#print(tactus_beats)
#print(tatum_beats)
#print(meter_beats)

In [512]:
# read audio file
kick = AudioSegment.from_wav('kick_test.wav')
snare = AudioSegment.from_wav('snare.wav')
hat = AudioSegment.from_wav('hh.wav')
# generate empty segment
meter_segment = AudioSegment.empty()
tactus_segment = AudioSegment.empty()
tatum_segment = AudioSegment.empty()

In [513]:
# --- Onset distribution 
# for the tactus layer
for idx, tactus in tactus_beats:
    # sample from binomial whether an onset takes place at the beat position
    onset = np.random.binomial(1, UP[int(idx)-1])
    if onset:
        # slice the sample according to the tactus duration
        sample = snare[:tactus] + AudioSegment.silent(duration=tactus-(snare.duration_seconds*1000))
    elif not onset:
        sample = AudioSegment.silent(duration=tactus)
        
    # concatenate the slices
    tactus_segment += sample
    
# for the tatum layer
for idx, tatum in tatum_beats:
    # sample from binomial whether an onset takes place at the beat position
    onset = np.random.binomial(1, LP[int(idx)-1])
    if onset:
        # slice the sample according to the tactus duration
        sample = hat[:tatum] + AudioSegment.silent(duration=tatum-(hat.duration_seconds*1000))
    elif not onset:
        sample = AudioSegment.silent(duration=tatum)
        
    # concatenate the slices
    tatum_segment += sample
    

# for the meter layer
for meter in meter_beats:
    # sample from binomial whether an onset takes place at the beat position
    onset = np.random.binomial(1, MP)
    if onset:
        # slice the sample according to the tactus duration
        sample = kick + AudioSegment.silent(duration=meter-(kick.duration_seconds*1000))
    elif not onset:
        sample = AudioSegment.silent(duration=meter)
        
    # concatenate the slices
    meter_segment += sample

In [506]:
output = meter_segment.overlay(tactus_segment).overlay(tatum_segment)

In [514]:
# add a chaotic layer
pips = np.round((output.duration_seconds*1000)/(ioi/8)).astype(int)
print('%d fine subdivisions' % pips)
# sample probability of onset for pips
onsets_on_pips = [np.random.binomial(1, 0.1) for x in range(pips)]
# use strange percussion
impulse = AudioSegment.from_wav('perc.wav').high_pass_filter(5000).normalize()
chaotic_segment = AudioSegment.empty()
for pip in onsets_on_pips:
    if pip:
        sample = impulse[:ioi/8]
    elif not pip:
        sample = AudioSegment.silent(duration=ioi/8)
        
    # concatenate the slices
    chaotic_segment += sample

295 fine subdivisions


In [515]:
output.overlay(chaotic_segment)